# Tải thư viện

In [1]:
!pip install huggingface_hub transformers
!pip install flash-attn
!pip install accelerate -q
!pip install bitsandbytes -q
!pip install peft trl
!pip install -q datasets
!pip install evaluate
!pip install tqdm
!pip install nltk
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.4 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187815463 sha256=d944fc7d2f962bce83fc4708c2fc0c21eaf8255962a0b350ae919362a51b7ef2
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn
  At

In [2]:
from datasets import load_dataset, DatasetDict, Features, Value
import re
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import torch
import accelerate 
import bitsandbytes 
from transformers import AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, AutoModelForCausalLM, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import numpy as np
import evaluate
import math
import string
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import json

2025-05-26 17:21:22.644641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748280082.882824      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748280082.946844      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Xử lý dataset

In [3]:
# # Load dataset từ hugging face
# dataset = load_dataset("isaacchung/hotpotqa-dev-raft-subset")

## LÀM SẠCH DỮ LIỆU 

In [4]:
# dataset = dataset.remove_columns([col for col in dataset.column_names['train'] if col not in ['question', 'cot_answer']])

# #  Xử lý để chỉ giữ lại nội dung sau <ANSWER>:
# def extract_answer(example):
#     match = re.search(r"<ANSWER>:\s*(.*)", example['cot_answer'], re.DOTALL)
#     if match:
#         example['cot_answer'] = match.group(1).strip()
#     else:
#         example['cot_answer'] = None 
#     return example

# dataset = dataset.map(extract_answer)

# # Lấy 4000 mẫu đầu tiên
# subset = dataset['train'].select(range(4000))

# # Chia thành 3000 mẫu train và 300 mẫu validation
# train_dataset = subset.select(range(3000))
# test_dataset = subset.select(range(3700, 4000))

# train_dataset = train_dataset.rename_column("cot_answer", "answer")
# test_dataset = test_dataset.rename_column("cot_answer", "answer")

# # Gộp thành DatasetDict
# final_dataset = DatasetDict({
#     'train': train_dataset,
#     'test': test_dataset
# })

# print(final_dataset)
# print(final_dataset['train'][0])

## CHUẨN HÓA FORMAT DỮ LIỆU CHO MODEL MISTRAL-7B 

In [5]:
# # Định nghĩa hàm format
# def format_prompt(example):
#     prompt = (
#         f"<s>[INST] {example['question']} [/INST] "
#         f"{example['answer']}</s>"
#     )
#     return {"text": prompt}

# # Áp dụng cho cả train và validation
# train_dataset = train_dataset.map(format_prompt)
# test_dataset = test_dataset.map(format_prompt)

# train_dataset = train_dataset.remove_columns(['question', 'answer'])
# test_dataset = test_dataset.remove_columns(['question', 'answer'])

# final_dataset = DatasetDict({
#     'train': train_dataset,
#     'test': test_dataset
# })

In [6]:
# print(final_dataset)
# print(final_dataset['train'][0])

# Load model Mistral-7B

In [7]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token=hf_token)

## Load model

In [8]:
# # --- Định nghĩa Model ID ---
# model_id = "mistralai/Mistral-7B-Instruct-v0.2"
# MAX_LENGTH = 512

# # --- Định nghĩa Cấu hình Lượng tử hóa (4-bit NF4) ---
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# # --- Load model và chia thiết bị ---
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=bnb_config,
#     device_map=0
# )

# # --- Chuẩn bị cho QLoRA training ---
# model = prepare_model_for_kbit_training(model)

# # --- Tải Tokenizer ---
# tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
# tokenizer.pad_token = tokenizer.eos_token

# def tokenize(examples):
#     texts = examples["text"]
#     input_ids_list = []
#     attention_masks = []
#     labels_list = []

#     for text in texts:
#         # Parse thủ công: tách phần instruction và output
#         if "[INST]" not in text or "[/INST]" not in text:
#             raise ValueError("Prompt không đúng định dạng [INST]...[/INST]")

#         instr_start = text.find("[INST]") + len("[INST]")
#         instr_end = text.find("[/INST]")
#         instruction = text[instr_start:instr_end].strip()
#         output = text[instr_end + len("[/INST]"):].strip()

#         full_prompt = f"<s>[INST] {instruction} [/INST] {output}</s>"

#         tokenized = tokenizer(
#             full_prompt,
#             truncation=True,
#             max_length=MAX_LENGTH,
#             padding="max_length"
#         )

#         input_ids = tokenized["input_ids"]
#         attention_mask = tokenized["attention_mask"]

#         # Chỉ huấn luyện phần trả lời: mask phần [INST] ... [/INST]
#         prompt_only = f"<s>[INST] {instruction} [/INST]"
#         prompt_ids = tokenizer(prompt_only, add_special_tokens=False)["input_ids"]
#         prompt_len = len(prompt_ids)

#         labels = [
#             tok_id if i >= prompt_len and tok_id != tokenizer.pad_token_id else -100
#             for i, tok_id in enumerate(input_ids)
#         ]

#         input_ids_list.append(input_ids)
#         attention_masks.append(attention_mask)
#         labels_list.append(labels)

#     return {
#         "input_ids": input_ids_list,
#         "attention_mask": attention_masks,
#         "labels": labels_list
#     }

# # --- Mapping dữ liệu ---
# tokenized_dataset = final_dataset.map(tokenize, batched=True)
# tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# # --- Kiểm tra ---
# print("✅ Mistral-7B-Instruct đã được tải thành công!")


## Lora

In [9]:
# peft_config = LoraConfig(
#     r=8,
#     lora_alpha=16,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = get_peft_model(model, peft_config)

## Cấu hình training

In [10]:
# # === Training arguments ===
# training_args = TrainingArguments(
#     output_dir="./qlora-mistral",
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=4,
#     num_train_epochs=3,
#     logging_steps=10,
#     learning_rate=2e-4,
#     bf16=True,
#     save_strategy="epoch",
#     report_to="none",
#     run_name="qlora-mistral",
#     save_total_limit=2,
# )

# # === Data collator ===
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [11]:
# # === Trainer ===
# trainer = Trainer(
#     model=model,
#     train_dataset=tokenized_dataset['train'],
#     eval_dataset=tokenized_dataset['test'],
#     args=training_args,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

In [12]:
# # === Train ===
# trainer.train()

# # === Save LoRA adapter ===
# model.save_pretrained("./qlora-mistral-adapter")
# tokenizer.save_pretrained("./qlora-mistral-adapter")

In [13]:
# logs = trainer.state.log_history
# train_loss = [log["loss"] for log in logs if "loss" in log]
# steps = [i * training_args.logging_steps for i in range(len(train_loss))]

# plt.plot(steps, train_loss, label="Training Loss")
# plt.xlabel("Steps")
# plt.ylabel("Loss")
# plt.title("Training Loss over Time")
# plt.legend()
# plt.grid(True)
# plt.show()

# Đánh giá model

## Load model pretrain

In [14]:
# Đường dẫn đến mô hình đã lưu trên Kaggle
adapter_path = "/kaggle/input/mistral-7b/transformers/default/1/qlora-mistral-adapter"

# Cấu hình lượng tử hóa 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load lại base model với lượng tử hóa 4bit
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=bnb_config,
    device_map="auto",
)

# Load lại adapter đã fine-tune
model = PeftModel.from_pretrained(base_model, adapter_path)

# Load tokenizer từ adapter
tokenizer = AutoTokenizer.from_pretrained(adapter_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [15]:
# MAX_LENGTH = 512

# def tokenize(examples):
#     texts = examples["text"]
#     input_ids_list = []
#     attention_masks = []
#     labels_list = []

#     for text in texts:
#         # Parse thủ công: tách phần instruction và output
#         if "[INST]" not in text or "[/INST]" not in text:
#             raise ValueError("Prompt không đúng định dạng [INST]...[/INST]")

#         instr_start = text.find("[INST]") + len("[INST]")
#         instr_end = text.find("[/INST]")
#         instruction = text[instr_start:instr_end].strip()
#         output = text[instr_end + len("[/INST]"):].strip()

#         full_prompt = f"<s>[INST] {instruction} [/INST] {output}</s>"

#         tokenized = tokenizer(
#             full_prompt,
#             truncation=True,
#             max_length=MAX_LENGTH,
#             padding="max_length"
#         )

#         input_ids = tokenized["input_ids"]
#         attention_mask = tokenized["attention_mask"]

#         # Chỉ huấn luyện phần trả lời: mask phần [INST] ... [/INST]
#         prompt_only = f"<s>[INST] {instruction} [/INST]"
#         prompt_ids = tokenizer(prompt_only, add_special_tokens=False)["input_ids"]
#         prompt_len = len(prompt_ids)

#         labels = [
#             tok_id if i >= prompt_len and tok_id != tokenizer.pad_token_id else -100
#             for i, tok_id in enumerate(input_ids)
#         ]

#         input_ids_list.append(input_ids)
#         attention_masks.append(attention_mask)
#         labels_list.append(labels)

#     return {
#         "input_ids": input_ids_list,
#         "attention_mask": attention_masks,
#         "labels": labels_list
#     }

# tokenized_dataset = final_dataset.map(tokenize, batched=True)
# tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

## Đánh giá model với EM

In [16]:
def normalize_answer(s):
    """Chuẩn hóa chuỗi để so sánh EM công bằng."""
    def remove_punctuation(text):
        return text.translate(str.maketrans('', '', string.punctuation))

    def lower(text):
        return text.lower()

    def white_space_fix(text):
        return ' '.join(text.split())

    return white_space_fix(remove_punctuation(lower(s)))

In [17]:
def extract_prompt_and_answer(text):
    answer_prefix = "<ANSWER>:"
    answer_start = text.find(answer_prefix)
    if answer_start == -1:
        raise ValueError("Không tìm thấy phần '<ANSWER>:' trong text.")

    prompt = text[:answer_start].strip()
    answer = text[answer_start + len(answer_prefix):].strip()
    return prompt, answer

In [18]:
def compute_f1(pred, gold):
    pred_tokens = normalize_answer(pred).split()
    gold_tokens = normalize_answer(gold).split()
    common = set(pred_tokens) & set(gold_tokens)
    num_same = sum(min(pred_tokens.count(token), gold_tokens.count(token)) for token in common)

    if len(pred_tokens) == 0 or len(gold_tokens) == 0:
        return int(pred_tokens == gold_tokens)
    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    return 2 * precision * recall / (precision + recall)

In [19]:
def compute_em_score(model, tokenizer, dataset):
    em_scores,f1s = [],[]

    for sample in dataset:
        # full_text = sample["text"]
        full_text = sample["text"]
        prompt, ground_truth = extract_prompt_and_answer(full_text)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=400,
                temperature=0.7,
                top_p=0.9,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Tách phần sinh ra sau "### Answer:"
        answer_start = generated_text.find("<ANSWER>:")
        if answer_start != -1:
            # Chỉ tìm <STOP> sau <ANSWER>:
            answer_end = generated_text.find("<STOP>", answer_start)
        
            if answer_end != -1:
                predicted_answer = generated_text[answer_start + len("<ANSWER>:"):answer_end].strip()
            else:
                predicted_answer = generated_text[answer_start + len("<ANSWER>:"):].strip()
        else:
            predicted_answer = generated_text.strip()

        # So sánh exact match (không phân biệt hoa thường)
        # em = int(predicted_answer.lower() == ground_truth.lower())
        em = int(normalize_answer(predicted_answer) == normalize_answer(ground_truth))
        f1 = compute_f1(predicted_answer, ground_truth)
        em_scores.append(em)
        f1s.append(f1)
    print("\n📊 EM avg:", sum(em_scores) / len(em_scores))
    print("📊 F1 avg:", sum(f1s)/len(f1s))  

In [20]:
def load_jsonl_with_inst(file_path):
    test_data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for idx, line in enumerate(f):
            item = json.loads(line)
            prompt = f'<s>[INST]{item["prompt"]}[/INST]'
            completion = item.get("completion", "").lstrip("$").strip()
            text = f"{prompt}\n<ANSWER>: {completion}"
            test_data.append({"text": text})

            # In ra 3 mẫu đầu để kiểm tra
            if idx < 2:
                print(f"Sample {idx+1}:\n{text}\n{'-'*40}")

    return test_data

test_data = load_jsonl_with_inst("/kaggle/input/converted-data-eval/converted_data_eval.jsonl")

Sample 1:
<s>[INST]<DOCUMENT>['Carlos Salazar (born 1933) is a Filipino actor who began his career with LVN Pictures and made several hit movies.', ' After making some 15 movies on the said studio, Salazar made one movie under Champion Pictures entitled "" and one movie under Larry Santiago Production for "Student Canteen".']</DOCUMENT>
<DOCUMENT>['Kevin Ogle is a news anchor for KFOR-TV (channel 4), the NBC affiliate in Oklahoma City, Oklahoma.', ' Kevin anchors the station\'s weeknight 6 and 10 p.m. newscasts with Linda Cavanaugh, does occasional reports including the "Bottom Line" financial segments seen during the station\'s 6:30\xa0p.m. newscast, and occasionally fills in as anchor of the 9 p.m. newscast on Independent station KAUT-TV.', ' Ogle also serves as moderator of the locally-produced Sunday morning political affairs talk show "Flashpoint", alongside panelists Mike Turpen and former Oklahoma City mayor Kirk Humphreys.']</DOCUMENT>
<DOCUMENT>['Abraham is a surname.', ' It c

In [21]:
compute_em_score(model, tokenizer, test_data)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



📊 EM avg: 0.0
📊 F1 avg: 0.015381174438091762


## Thử nghiệm trên vài mẫu dữ liệu

In [22]:
# prompts = [
#     "What year did the film Snow premiere on ABC?",
#     "Who plays PI Bobbi Bacha in Suburban Madness?",
#     "How many albums has Jimmy Bosch recorded as a solo artist?",
#     "What are the basic characteristics of elite theory?",
#     "How many minutes of live footage were included in the episode?"
# ]


# for i, question in enumerate(prompts):
#     print(f"\n================= Sample {i+1} =================")

#     # Format prompt theo chuẩn Mistral
#     prompt = f"<s>[INST] {question} [/INST]"

#     # Tokenize và chuyển sang GPU (nếu có)
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

#     # Sinh câu trả lời
#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=150,
#             temperature=0.7,
#             top_p=0.9,
#             do_sample=True,
#             pad_token_id=tokenizer.eos_token_id
#         )

#     # Giải mã output
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Trích xuất phần trả lời (sau thẻ [/INST])
#     answer_start = response.find("[/INST]")
#     if answer_start != -1:
#         answer_text = response[answer_start + len("[/INST]"):].strip()

#         # Dừng lại nếu mô hình sinh thêm prompt khác
#         next_inst = answer_text.find("[INST]")
#         if next_inst != -1:
#             answer_text = answer_text[:next_inst].strip()
#     else:
#         answer_text = response.strip()

#     # In kết quả
#     print(answer_text)